In [2]:
#Imports
import math
import requests
import json
import csv
import pandas as pd
import requests
from requests.exceptions import HTTPError
import re

## Funcion para limpiar strings

In [3]:
def limpiar(texto):
    return re.sub(r'[^A-Za-z0-9]', '', texto)
def limpiarDbpedia(texto):
    return texto.replace("http://dbpedia.org/resource/", "").replace("_", " ")

## Datasets

In [30]:
data = pd.read_csv("ScopusSearch.csv")
data

In [31]:
data['coverDate'] = pd.to_datetime(data['coverDate'], format='%Y-%m-%d')
data['año'] = data['coverDate'].dt.year
len(data)

In [32]:
import pandas as pd
import matplotlib.pyplot as plt

# Obtener el conteo de filas por año
conteo_por_año = data['año'].value_counts().sort_index()

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))
conteo_por_año.plot(kind='bar', color='skyblue')

# Agregar los valores al final de cada barra de manera horizontal
for p in plt.gca().patches:
    height = p.get_height()
    plt.text(p.get_x() + p.get_width() / 2, height + 0.05 * height,  # Posición del texto
             int(height),  # El valor que se mostrará
             ha='center',  # Alinear horizontalmente al centro
             va='bottom')  # Alinear verticalmente al final

plt.title('Publicaciones por año')
plt.xlabel('Año')
plt.ylabel('Cantidad')
plt.show()


In [33]:
data1 = pd.read_csv("AbstractRetrieval.csv")
data1

In [34]:
scimagojr = pd.read_csv("123.csv")
journalOriginal = data[["eid","source_id"]]
merged_journal = pd.merge(journalOriginal, scimagojr, how='left', left_on='source_id', right_on='Sourceid')
merged_journal = merged_journal[["eid","source_id","Publisher","Issn","Rank","Title","SJR",
                                 "H index","Coverage","SJR Best Quartile","Categories","Region","Country"]]
merged_journal

In [35]:
mergeData = pd.merge(data, data1, on='eid', how='left')
mergeData

In [36]:
referen = pd.read_csv("references.csv")
referen = pd.merge(data['eid'], referen, on='eid', how='right')
referen

In [37]:
journ = merged_journal.fillna(0)
journ.rename(columns={'H index': 'hIndex'}, inplace=True)
journ.rename(columns={'SJR Best Quartile': 'SJRBestQuartile'}, inplace=True)
journ.head(2)

In [4]:
triplesLLm = pd.read_csv("triplesLLMcompletas.csv")
triplesLLm.rename(columns={'tail': 'tail1'}, inplace=True)
triplesLLm = triplesLLm.dropna(subset=['tail1'])


## Creacion de csv para NEO4J
## Papers

In [39]:
papers = mergeData[["eid","doi","title","subtypeDescription","citedby_count","url","description"]]
papers.columns = ['identifier', 'doi', 'title', 'publicationType', 'citation', 'url',"description"]
papers.to_csv('csvNEO4J/Nodos/papers.csv', index=False)
papers

## KeyWords

## Publisher

In [102]:
keyWords = mergeData[["eid","authkeywords","idxterms"]]
keyWords = keyWords.fillna(0)
paperKey = []
Key = []
for c in keyWords.itertuples():
    if c.authkeywords != 0:
        h = c.authkeywords.split(' | ')
        for i in h:
            paperKey.append([c.eid,limpiar(i)]) 
            Key.append([limpiar(i),i,"authorKeywords"])
    
    if c.idxterms != 0:
        paperKey.append([c.eid,limpiar(c.idxterms[2:-2])])
        Key.append([limpiar(c.idxterms[2:-2]),c.idxterms[2:-2],"idxterms"])
        
        
paperKey = pd.DataFrame(paperKey, columns=['papers', 'key']).drop_duplicates()
paperKey.to_csv('csvNEO4J/Relaciones/papers-key.csv', index=False)
           
Key = pd.DataFrame(Key, columns=['identifier', 'name','contentType']).drop_duplicates()
Key.to_csv('csvNEO4J/Nodos/key.csv', index=False)

In [103]:
keyWords1 = keyWords[keyWords['idxterms'] != 0]
keyWords1

In [104]:
paperPublisher = []
publisher1 = []
publisher = mergeData[["eid","publisher"]].fillna(0)
for c in publisher.itertuples():
    if c.publisher != 0:
        paperPublisher.append([c.eid,limpiar(c.publisher)])
        publisher1.append([limpiar(c.publisher),c.publisher])
        
paperPublisher1 = pd.DataFrame(paperPublisher, columns=['paper', 'publisher']).drop_duplicates()
paperPublisher1.to_csv('csvNEO4J/Relaciones/paperPublisher.csv', index=False)

publisher2 = pd.DataFrame(publisher1, columns=['identifier', 'name']).drop_duplicates()
publisher2.to_csv('csvNEO4J/Nodos/publisher.csv', index=False)

## References

In [45]:
ref = []
paper2 = []
referen1 = referen.dropna(subset=[ 'title'])

for c in referen1.itertuples():
    ref.append([c.eid,"2-s2.0-"+str(c.id)])
    paper2.append(["2-s2.0-"+str(c.id),c.doi,c.title])
    

ref = pd.DataFrame(ref, columns=['paper1', 'paper2']).drop_duplicates()
ref.to_csv('csvNEO4J/Relaciones/references.csv', index=False)    


# Asegúrate de que ambos DataFrames tengan la columna 'identificador'

paper2 = pd.DataFrame(paper2, columns=['identifier','doi','title']).drop_duplicates(subset=['identifier'])
paper2 = paper2[~paper2['identifier'].isin(papers['identifier'])]
paper2.to_csv('csvNEO4J/Nodos/papers2.csv', index=False) 
paper2


## Journal / Country / Region / Concept

In [108]:
paperJournal = []
journal = []
journalPublisher = []
journalCountry = []
country = []
regionCountry = []
region = []
journalConcept = []
Concept = []
for c in journ.itertuples():
    if c.Publisher != 0:       
        paperJournal.append([c.eid,c.source_id])
        
        journal.append([c.source_id,c.Issn,c.Rank,c.Title,c.SJR,c.hIndex,c.Coverage,c.SJRBestQuartile])
        
        journalPublisher.append([c.source_id,limpiar(c.Publisher)])
        
        journalCountry.append([c.source_id,limpiar(c.Country)])
        
        country.append([limpiar(c.Country),c.Country])
        
        regionCountry.append([limpiar(c.Region),limpiar(c.Country)])
        
        region.append([limpiar(c.Region),c.Region])

        g = c.Categories.split(', ')

        for i in g:
            journalConcept.append([c.source_id,limpiar(i.split(' (')[0])])
            Concept.append([limpiar(i.split(' (')[0]),i.split(' (')[0]])
        
paperJournal = pd.DataFrame(paperJournal, columns=['paper', 'journal']).drop_duplicates()
paperJournal.to_csv('csvNEO4J/Relaciones/paperJournal.csv', index=False)    

journal = pd.DataFrame(journal, columns=['identifier', 'issn','rank','title','sjr','hIndex','coverage','bestQuartile']).drop_duplicates()
journal.to_csv('csvNEO4J/Nodos/Journal.csv', index=False)    

journalPublisher = pd.DataFrame(journalPublisher, columns=['journal', 'publisher']).drop_duplicates()
journalPublisher.to_csv('csvNEO4J/Relaciones/journalPublisher.csv', index=False)   

journalCountry = pd.DataFrame(journalCountry, columns=['journal', 'country']).drop_duplicates()
journalCountry.to_csv('csvNEO4J/Relaciones/journalCountry.csv', index=False)   

# Se añaden en la parte de affiliation, para crear un solo archivo
# country = pd.DataFrame(country, columns=['identifier', 'name']).drop_duplicates()
# country.to_csv('csvNEO4J/country.csv', index=False)   

regionCountry = pd.DataFrame(regionCountry, columns=['region', 'country']).drop_duplicates()
regionCountry.to_csv('csvNEO4J/Relaciones/regionCountry.csv', index=False)   

region = pd.DataFrame(region, columns=['identifier', 'name']).drop_duplicates()
region.to_csv('csvNEO4J/Nodos/region.csv', index=False) 

journalConcept = pd.DataFrame(journalConcept, columns=['journal', 'concept']).drop_duplicates()
journalConcept.to_csv('csvNEO4J/Relaciones/journalConcept.csv', index=False) 

Concept = pd.DataFrame(Concept, columns=['identifier', 'name']).drop_duplicates()
Concept.to_csv('csvNEO4J/Nodos/Concept.csv', index=False) 


## Autor 

In [109]:
dataAu = data.fillna(0)

papersAutores = []
autores = []

autoresAff = []

for c in dataAu.itertuples():

    auID = c.author_ids.split(';')
    auName = c.author_names.split(';')
    if c.author_afids != 0:
        authorAfids = [afid.strip() for afid in c.author_afids.split(';') if afid.strip()]

    
    for i in range(len(auID)):
        papersAutores.append([c.eid,auID[i]])
        autores.append([auID[i],auName[i]])
        
        if c.author_afids != 0:
            aff = authorAfids[0].split('-')
            for g in aff:
                autoresAff.append([auID[i],g]) 
                
papersAutores = pd.DataFrame(papersAutores, columns=['paper', 'autor']).drop_duplicates()
papersAutores.to_csv('csvNEO4J/Relaciones/papersAutores.csv', index=False) 

autores = pd.DataFrame(autores, columns=['identifier', 'name']).drop_duplicates()
autores.to_csv('csvNEO4J/Nodos/autores.csv', index=False) 

autoresAff = pd.DataFrame(autoresAff, columns=['autor', 'affiliation']).drop_duplicates()
autoresAff.to_csv('csvNEO4J/Relaciones/autoresAffiliations.csv', index=False) 


## Affiliation/City/Country

In [110]:
aff = []
affCity = []
city = []
cityCountry = []

for c in dataAu.itertuples():
    if c.afid != 0:
        afid = c.afid.split(';')
        afidName = c.affilname.split(';')
        if c.affiliation_city != 0:
            afCity = c.affiliation_city.split(';')
            afCountry = c.affiliation_country.split(';')
            
        for i in range(len(afid)):
            aff.append([afid[i],afidName[i]])
            if c.affiliation_city != 0:
                affCity.append([afid[i],limpiar(afCity[i])])
                city.append([limpiar(afCity[i]),afCity[i]])
                cityCountry.append([limpiar(afCity[i]),limpiar(afCountry[i])])
                country.append([limpiar(afCountry[i]),afCountry[i]])

aff = pd.DataFrame(aff, columns=['identifier', 'name']).drop_duplicates()
aff.to_csv('csvNEO4J/Nodos/affiliation.csv', index=False) 

affCity = pd.DataFrame(affCity, columns=['affili', 'city']).drop_duplicates()
affCity.replace('', pd.NA, inplace=True)
affCity = affCity.dropna()
affCity.to_csv('csvNEO4J/Relaciones/affCity.csv', index=False)

city = pd.DataFrame(city, columns=['identifier', 'name']).drop_duplicates()
city = city[~(city == '').any(axis=1)]
city.to_csv('csvNEO4J/Nodos/city.csv', index=False) 

cityCountry = pd.DataFrame(cityCountry, columns=['city', 'country']).drop_duplicates()
cityCountry = cityCountry[~(cityCountry == '').any(axis=1)]
cityCountry.to_csv('csvNEO4J/Relaciones/cityCountry.csv', index=False)

country = pd.DataFrame(country, columns=['identifier', 'name']).drop_duplicates()
country.to_csv('csvNEO4J/Nodos/country.csv', index=False) 

## Chunks / Fact

In [5]:
paperChunk = []   #
Chunk = []        #

chunkFact = []    #

Fact = []         

Algoritm = []
FactAlg = []
FactAlgT = []

DataSet = []
FactDat = []
FactDatT = []

Evaluation = []
FactEva = []
FactEvaT = []

Exprement = []
FactExp = []
FactExpT = []


Group = []
FactGro = []
FactGroT = []


Method = []
FactMet = []
FactMetT = []


Person = []
FactPer = []
FactPerT = []


ResearchField = []
FactRes = []
FactResT = []

Technology = []
FactTec = []
FactTecT = []


Tool = []
FactToo = []
FactTooT = []

In [6]:
l = len(triplesLLm) -1
i = 1
j = 1
last = 0
HI = len(triplesLLm)
for idx in range (HI):
    c = triplesLLm.iloc[idx]
    chunkFact.append([c.eid +"_chunk"+str(i),c.eid +"_chunk"+str(i)+"_fact"+str(j)])
    Fact.append([c.eid +"_chunk"+str(i)+"_fact"+str(j),c.head1,c.relation,c.tail1])
    if(c.head1.startswith("http")):
        match c.head_type:
            case "Algoritm":
                Algoritm.append([limpiar(c.head1),c.head1,limpiarDbpedia(c.head1)])
                FactAlg.append([c.eid +"_chunk"+str(i)+"_fact"+str(j),limpiar(c.head1),])
                
            case "DataSet":
                DataSet.append([limpiar(c.head1),c.head1,limpiarDbpedia(c.head1)])
                FactDat.append([c.eid +"_chunk"+str(i)+"_fact"+str(j),limpiar(c.head1)])

                
            case "Evaluation":
                Evaluation.append([limpiar(c.head1),c.head1,limpiarDbpedia(c.head1)])
                FactEva.append([c.eid +"_chunk"+str(i)+"_fact"+str(j),limpiar(c.head1)])

                
            case "Exprement":
                Exprement.append([limpiar(c.head1),c.head1,limpiarDbpedia(c.head1)])
                FactExp.append([c.eid +"_chunk"+str(i)+"_fact"+str(j),limpiar(c.head1)])
                
            case "Group":
                Group.append([limpiar(c.head1),c.head1,limpiarDbpedia(c.head1)])
                FactGro.append([c.eid +"_chunk"+str(i)+"_fact"+str(j),limpiar(c.head1)])
                
            case "Method":
                Method.append([limpiar(c.head1),c.head1,limpiarDbpedia(c.head1)])
                FactMet.append([c.eid +"_chunk"+str(i)+"_fact"+str(j),limpiar(c.head1)])
                
            case "Person":
                Person.append([limpiar(c.head1),c.head1,limpiarDbpedia(c.head1)])
                FactPer.append([c.eid +"_chunk"+str(i)+"_fact"+str(j),limpiar(c.head1)])
                
            case "ResearchField":
                ResearchField.append([limpiar(c.head1),c.head1,limpiarDbpedia(c.head1)])
                FactRes.append([c.eid +"_chunk"+str(i)+"_fact"+str(j),limpiar(c.head1)])
                
            case "Technology":
                Technology.append([limpiar(c.head1),c.head1,limpiarDbpedia(c.head1)])
                FactTec.append([c.eid +"_chunk"+str(i)+"_fact"+str(j),limpiar(c.head1)])

                
            case "Tool":
                Tool.append([limpiar(c.head1),c.head1,limpiarDbpedia(c.head1)])
                FactToo.append([c.eid +"_chunk"+str(i)+"_fact"+str(j),limpiar(c.head1)])
                
                
    if(c.tail1.startswith("http")):
        match c.tail_type:
            case "Algoritm":
                FactAlgT.append([c.eid +"_chunk"+str(i)+"_fact"+str(j),limpiar(c.tail1)])
                
            case "DataSet":
                FactDatT.append([c.eid +"_chunk"+str(i)+"_fact"+str(j),limpiar(c.tail1)])

            case "Evaluation":
                FactEvaT.append([c.eid +"_chunk"+str(i)+"_fact"+str(j),limpiar(c.tail1)])
                
            case "Exprement":
                FactExpT.append([c.eid +"_chunk"+str(i)+"_fact"+str(j),limpiar(c.tail1)])
                
            case "Group":
                FactGroT.append([c.eid +"_chunk"+str(i)+"_fact"+str(j),limpiar(c.tail1)])
                
            case "Method":
                FactMetT.append([c.eid +"_chunk"+str(i)+"_fact"+str(j),limpiar(c.tail1)])
                
            case "Person":
                FactPerT.append([c.eid +"_chunk"+str(i)+"_fact"+str(j),limpiar(c.tail1)])
                
            case "ResearchField":
                FactResT.append([c.eid +"_chunk"+str(i)+"_fact"+str(j),limpiar(c.tail1)])
                
            case "Technology":
                FactTecT.append([c.eid +"_chunk"+str(i)+"_fact"+str(j),limpiar(c.tail1)])
    
            case "Tool":
                FactTooT.append([c.eid +"_chunk"+str(i)+"_fact"+str(j),limpiar(c.tail1)])
        
    j += 1        
    if (idx != l):
        cSig = triplesLLm.iloc[idx+1]
        if (c.eid == cSig.eid and c.id != cSig.id):
            Chunk.append([c.eid +"_chunk"+str(i),c.content,c.vector,c.word_count,c.eid +"_chunk"+str(i+1)])
            paperChunk.append([c.eid,c.eid +"_chunk"+str(i)])
            i += 1
            j = 1

        elif (c.id != cSig.id):
            Chunk.append([c.eid +"_chunk"+str(i),c.content,c.vector,c.word_count])
            paperChunk.append([c.eid,c.eid +"_chunk"+str(i)])
            i = 1

    else:
        Chunk.append([c.eid +"_chunk"+str(i),c.content,c.vector,c.word_count])
        paperChunk.append([c.eid,c.eid +"_chunk"+str(i)])




In [7]:
paperChunk = pd.DataFrame(paperChunk, columns=['paper', 'chunk']).drop_duplicates()
paperChunk.to_csv('csvNEO4J/Relaciones/paperChunk.csv', index=False)

Chunk = pd.DataFrame(Chunk, columns=['identifier', 'content','vector','word_count','next']).drop_duplicates()
Chunk.to_csv('csvNEO4J/Nodos/chunk.csv', index=False) 

chunkFact = pd.DataFrame(chunkFact, columns=['chunk', 'fact']).drop_duplicates()
chunkFact.to_csv('csvNEO4J/Relaciones/chunkFact.csv', index=False)

Fact = pd.DataFrame(Fact, columns=['identifier', 'hasSubject','hasProperty','hasObject']).drop_duplicates()
Fact.to_csv('csvNEO4J/Nodos/fact.csv', index=False) 


Algoritm = pd.DataFrame(Algoritm, columns=['identifier', 'uri','label']).drop_duplicates()
Algoritm.to_csv('csvNEO4J/Nodos/algoritm.csv', index=False)   
FactAlg = pd.DataFrame(FactAlg, columns=['fact', 'algoritm']).drop_duplicates()
FactAlg.to_csv('csvNEO4J/Relaciones/FactAlg.csv', index=False)
FactAlgT = pd.DataFrame(FactAlgT, columns=['fact', 'algoritm']).drop_duplicates()
FactAlgT.to_csv('csvNEO4J/Relaciones/FactAlgT.csv', index=False)

DataSet = pd.DataFrame(DataSet, columns=['identifier', 'uri','label']).drop_duplicates()
DataSet.to_csv('csvNEO4J/Nodos/dataSet.csv', index=False)   
FactDat = pd.DataFrame(FactDat, columns=['fact', 'dataSet']).drop_duplicates()
FactDat.to_csv('csvNEO4J/Relaciones/factDat.csv', index=False)
FactDatT = pd.DataFrame(FactDatT, columns=['fact', 'dataSet']).drop_duplicates()
FactDatT.to_csv('csvNEO4J/Relaciones/factDatT.csv', index=False)

Evaluation = pd.DataFrame(Evaluation, columns=['identifier', 'uri','label']).drop_duplicates()
Evaluation.to_csv('csvNEO4J/Nodos/evaluation.csv', index=False)   
FactEva = pd.DataFrame(FactEva, columns=['fact', 'evaluation']).drop_duplicates()
FactEva.to_csv('csvNEO4J/Relaciones/factEva.csv', index=False)
FactEvaT = pd.DataFrame(FactEvaT, columns=['fact', 'evaluation']).drop_duplicates()
FactEvaT.to_csv('csvNEO4J/Relaciones/factEvaT.csv', index=False)


Exprement = pd.DataFrame(Exprement, columns=['identifier', 'uri','label']).drop_duplicates()
Exprement.to_csv('csvNEO4J/Nodos/exprement.csv', index=False)   
FactExp = pd.DataFrame(FactExp, columns=['fact', 'exprement']).drop_duplicates()
FactExp.to_csv('csvNEO4J/Relaciones/factExp.csv', index=False)
FactExpT = pd.DataFrame(FactExpT, columns=['fact', 'exprement']).drop_duplicates()
FactExpT.to_csv('csvNEO4J/Relaciones/factExpT.csv', index=False)

Group = pd.DataFrame(Group, columns=['identifier', 'uri','label']).drop_duplicates()
Group.to_csv('csvNEO4J/Nodos/group.csv', index=False)   
FactGro = pd.DataFrame(FactGro, columns=['fact', 'group']).drop_duplicates()
FactGro.to_csv('csvNEO4J/Relaciones/factGro.csv', index=False)
FactGroT = pd.DataFrame(FactGroT, columns=['fact', 'group']).drop_duplicates()
FactGroT.to_csv('csvNEO4J/Relaciones/factGroT.csv', index=False)

Method = pd.DataFrame(Method, columns=['identifier', 'uri','label']).drop_duplicates()
Method.to_csv('csvNEO4J/Nodos/method.csv', index=False)   
FactMet = pd.DataFrame(FactMet, columns=['fact', 'method']).drop_duplicates()
FactMet.to_csv('csvNEO4J/Relaciones/factMet.csv', index=False)
FactMetT = pd.DataFrame(FactMetT, columns=['fact', 'method']).drop_duplicates()
FactMetT.to_csv('csvNEO4J/Relaciones/factMetT.csv', index=False)

Person = pd.DataFrame(Person, columns=['identifier', 'uri','label']).drop_duplicates()
Person.to_csv('csvNEO4J/Nodos/person.csv', index=False)   
FactPer = pd.DataFrame(FactPer, columns=['fact', 'person']).drop_duplicates()
FactPer.to_csv('csvNEO4J/Relaciones/factPer.csv', index=False)
FactPerT = pd.DataFrame(FactPerT, columns=['fact', 'person']).drop_duplicates()
FactPerT.to_csv('csvNEO4J/Relaciones/factPert.csv', index=False)


ResearchField = pd.DataFrame(ResearchField, columns=['identifier', 'uri','label']).drop_duplicates()
ResearchField.to_csv('csvNEO4J/Nodos/researchField.csv', index=False)   
FactRes = pd.DataFrame(FactRes, columns=['fact', 'researchField']).drop_duplicates()
FactRes.to_csv('csvNEO4J/Relaciones/factRes.csv', index=False)
FactResT = pd.DataFrame(FactResT, columns=['fact', 'researchField']).drop_duplicates()
FactResT.to_csv('csvNEO4J/Relaciones/factResT.csv', index=False)

Technology = pd.DataFrame(Technology, columns=['identifier', 'uri','label']).drop_duplicates()
Technology.to_csv('csvNEO4J/Nodos/technology.csv', index=False)   
FactTec = pd.DataFrame(FactTec, columns=['fact', 'technology']).drop_duplicates()
FactTec.to_csv('csvNEO4J/Relaciones/factTec.csv', index=False)
FactTecT = pd.DataFrame(FactTecT, columns=['fact', 'technology']).drop_duplicates()
FactTecT.to_csv('csvNEO4J/Relaciones/factTecT.csv', index=False)

Tool = pd.DataFrame(Tool, columns=['identifier', 'uri','label']).drop_duplicates()
Tool.to_csv('csvNEO4J/Nodos/tool.csv', index=False)   
FactToo = pd.DataFrame(FactToo, columns=['fact', 'tool']).drop_duplicates()
FactToo.to_csv('csvNEO4J/Relaciones/factToo.csv', index=False)
FactTooT = pd.DataFrame(FactTooT, columns=['fact', 'tool']).drop_duplicates()
FactTooT.to_csv('csvNEO4J/Relaciones/factTooT.csv', index=False)


In [8]:
Technology

In [9]:
Chunk